In [19]:
def score_matrix(seq1,seq2):
    
    nrows = len(seq1)+1
    ncols = len(seq2)+1
    
    # initialize score matrix
    score_matrix = [[0 for x in range(ncols)] for x in range(nrows)]
    traceback_matrix = [[0 for x in range(ncols)] for x in range(nrows)]
    
    # calculate score matrix
    for i in range(nrows):
        for j in range(ncols):
            
            if i==j==0: # can optimize this slighty to not check condition
                score_matrix[i][j]=0
                traceback_matrix[i][j]="done"
            else:
                score_matrix[i][j],traceback_matrix[i][j] = calculate_score(score_matrix, seq1, seq2, i, j)
    
    return score_matrix,traceback_matrix
    
    
def calculate_score(score_matrix, seq1, seq2, i, j):
    """
    For the position in a global alignment matrix, i>0 and j>0
    +10 for match, -2 for mismatch, -5 for gap
    If you are in row 0, always return gap and left
    If you are in column zero, always return gap and up
    If not, calculate the score for the three boxes around the cell,
    pick the best one and return the appropriate value and traceback value
    """
    gap = -5
    match = 10
    mismatch = 2
    
    if i==0:
        return score_matrix[i][j-1] + gap,"left"
    elif j==0:
        return score_matrix[i-1][j] + gap,"up"
    else:
        from_diag = score_matrix[i-1][j-1] + (+match if seq1[i-1] == seq2[j-1] else -mismatch)
        from_up = score_matrix[i-1][j] + gap
        from_left = score_matrix[i][j-1] + gap
        
        values = [from_diag,from_up,from_left]
        options = ["diag","up","left"]
        index_max=max(range(len(values)),key=values.__getitem__)
        
        return values[index_max],options[index_max]
    
def traceback(traceback_matrix,seq1,seq2):
    """
    accepts a traceback matrix and two sequences from which it was derived
    and outputs an alignment
    seq1 is on the rows of the score matrix
    seq2 is on the columns of the score matrix
    """
    nrows = len(traceback_matrix)
    ncols = len(traceback_matrix[0])
    
    i,j = nrows-1,ncols-1
    aligned_seq1=[]
    aligned_seq2=[] ## indexing is shit here must fix
    
    while i>0 or j>0:
        if traceback_matrix[i][j]=="diag":
            aligned_seq1.insert(0,seq1[i-1])
            aligned_seq2.insert(0,seq2[j-1])
            i-=1
            j-=1
        elif traceback_matrix[i][j]=="left":
            aligned_seq1.insert(0,"-")
            aligned_seq2.insert(0,seq2[j-1])
            j-=1
        elif traceback_matrix[i][j]=="up":
            aligned_seq1.insert(0,seq1[i-1])
            aligned_seq2.insert(0,"-")
            i-=1
            
    return aligned_seq1,aligned_seq2
    

In [20]:
seq1="cattcac"
seq2="ctcgcagc"
a,b=score_matrix(seq1,seq2)

In [21]:
b

[['done', 'left', 'left', 'left', 'left', 'left', 'left', 'left', 'left'],
 ['up', 'diag', 'left', 'diag', 'left', 'diag', 'left', 'left', 'diag'],
 ['up', 'up', 'diag', 'diag', 'diag', 'diag', 'diag', 'left', 'left'],
 ['up', 'up', 'diag', 'left', 'left', 'left', 'up', 'diag', 'diag'],
 ['up', 'up', 'diag', 'diag', 'diag', 'diag', 'diag', 'diag', 'diag'],
 ['up', 'diag', 'up', 'diag', 'left', 'diag', 'left', 'left', 'diag'],
 ['up', 'up', 'up', 'up', 'diag', 'diag', 'diag', 'left', 'left'],
 ['up', 'diag', 'up', 'diag', 'diag', 'diag', 'up', 'diag', 'diag']]

In [22]:
traceback(b,seq1,seq2)

(['c', 'a', 't', '-', 't', 'c', 'a', '-', 'c'],
 ['c', '-', 't', 'c', 'g', 'c', 'a', 'g', 'c'])